# Dependencies

In [1]:
%load_ext autoreload
%autoreload 2
import nltk
import ipywidgets
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
tqdm.pandas()
# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True)
import torch
import datetime
import os
import datasets_handler
import evaluation_metrics
import zeroberto
from zeroberto import ZeroBERTo
# import datasets
# from datasets import Dataset
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
import gc

/Users/alealcoforado/Documents/Projetos/ZeroBERTo/zeroberto/datasets_handler.py:165: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def splitDocuments(docs: pd.Series()) -> list():
0it [00:00, ?it/s]


# Dataset

In [16]:
which_dataset = 'folhauol' 
# which_dataset = 'bbc-news'
# which_dataset = 'ag_news'
# which_dataset = 'imdb'

hyp_template = "{}"
# hyp_template = "Esse texto está relacionado a {}."
# hyp_template = "O tema principal deste texto é {}."
# hyp_template = "this text is about {}."
# hyp_template = "this article is about {}."

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)
classes_list = list(raw_data[class_col].unique())
print(classes_list)

['Poder e Política no Brasil', 'Mercado', 'Notícias de fora do Brasil', 'Esporte', 'Tecnologia', 'Meio Ambiente', 'Equilíbrio e Saúde', 'Educação', 'TV, Televisão e Entretenimento', 'Ciência', 'Turismo', 'Comida']


# ZeroBERTo

## Parameters

In [17]:
# max_inferences = len(train) #estava em 5000
max_inferences = 1000

# zeroshot_method = "probability_threshold"
# probability_goal = 0.9
# top_n_goal = 8

zeroshot_method = "dotproduct"  
# zeroshot_method = "kmeans"  

zeroshot_config = {
    'dataset':which_dataset,
    'class_col':class_col,
    'data_col':data_col,
    'split':"zeroshot",
    'method':zeroshot_method,
    # 'prob_goal':probability_goal,
    # 'top_n_goal':top_n_goal,
    'max_inferences':max_inferences,
    'classes':classes_list,#list(dict_classes_folha.values())
    'template': hyp_template,
    'random_state':422,
    # 'trainings_done':0
}
zeroshot_config

{'dataset': 'folhauol',
 'class_col': 'class',
 'data_col': 'text',
 'split': 'zeroshot',
 'method': 'dotproduct',
 'max_inferences': 1000,
 'classes': ['Poder e Política no Brasil',
  'Mercado',
  'Notícias de fora do Brasil',
  'Esporte',
  'Tecnologia',
  'Meio Ambiente',
  'Equilíbrio e Saúde',
  'Educação',
  'TV, Televisão e Entretenimento',
  'Ciência',
  'Turismo',
  'Comida'],
 'template': '{}',
 'random_state': 422}

In [4]:
train_data = raw_data.sample(zeroshot_config['max_inferences'],random_state=zeroshot_config['random_state']).sort_index()
len(train_data)

1000

## Model

In [53]:
zeroshot_config['similarity_model'] = 'sentence-transformers/stsb-xlm-r-multilingual'
zeroshot_config['setfit_model'] = 'ricardo-filho/bert-base-portuguese-cased-nli-assin-2'

model = ZeroBERTo(classes_list=zeroshot_config['classes'],hypothesis_template=zeroshot_config['template'],
                  train_dataset=train_data,labeling_method=zeroshot_config['method'],embeddingModel=zeroshot_config['similarity_model'])

In [12]:
model.embeddingModel

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

## Inference

In [13]:
results = zeroberto.runZeroberto(model,train_data[data_col].to_list(),zeroshot_config)  ##X## COMENTAR AQUI 

Preds: 100  - Total time: 7.13 seconds - ETA: 1.2 minutes
Preds: 200  - Total time: 14.2 seconds - ETA: 1.2 minutes
Preds: 300  - Total time: 21.32 seconds - ETA: 1.2 minutes
Preds: 400  - Total time: 28.37 seconds - ETA: 1.2 minutes
Preds: 500  - Total time: 35.44 seconds - ETA: 1.2 minutes
Preds: 600  - Total time: 42.46 seconds - ETA: 1.2 minutes
Preds: 700  - Total time: 49.57 seconds - ETA: 1.2 minutes
Preds: 800  - Total time: 56.98 seconds - ETA: 1.2 minutes
Preds: 900  - Total time: 63.8 seconds - ETA: 1.2 minutes
Preds: 1000  - Total time: 70.55 seconds - ETA: 1.2 minutes


## Evaluation

In [15]:
len(results)

1000

In [18]:
df_results = model.evaluateLabeling(results)

top 1: {'accuracy': 0.4166666666666667}
top 2: {'accuracy': 0.375}
top 3: {'accuracy': 0.3888888888888889}
top 4: {'accuracy': 0.40425531914893614}
top 5: {'accuracy': 0.39655172413793105}
top 6: {'accuracy': 0.391304347826087}
top 7: {'accuracy': 0.3625}
top 8: {'accuracy': 0.31868131868131866}
top 9: {'accuracy': 0.3137254901960784}
top 10: {'accuracy': 0.33035714285714285}
top 11: {'accuracy': 0.319672131147541}
top 12: {'accuracy': 0.3409090909090909}
top 13: {'accuracy': 0.3380281690140845}
top 14: {'accuracy': 0.3355263157894737}
top 15: {'accuracy': 0.32098765432098764}
top 16: {'accuracy': 0.30409356725146197}
top 1000: {'accuracy': 0.3}


In [19]:
try: print(zeroshot_metrics)
except: pass
zeroshot_metrics = evaluation_metrics.get_metrics(df_results['prediction_code'].to_list(),df_results['class_code'].to_list())
print(zeroshot_metrics)

{'weighted': [{'accuracy': 0.011}, {'precision': 0.002404475272890803}, {'recall': 0.011}, {'f1': 0.0018934333681255703}], 'macro': [{'accuracy': 0.011}, {'precision': 0.010743698414212582}, {'recall': 0.0973731884057971}, {'f1': 0.010337181335813345}]}
{'weighted': [{'accuracy': 0.3}, {'precision': 0.4301732537629209}, {'recall': 0.3}, {'f1': 0.22531228268187875}], 'macro': [{'accuracy': 0.3}, {'precision': 0.2734849261648797}, {'recall': 0.2971967721787631}, {'f1': 0.21590448967919532}]}


## Save Labeling Results

In [20]:
save_time = evaluation_metrics.saveZeroshotResults(zeroshot_config,df_results)

predictions_and_probabilities_test_2023_04_07__15_56_16.csv
zeroshot_config_test_2023_04_07__15_56_16.csv


# ST Training

In [21]:
# save_time = '2023_04_07__00_10_47'

In [22]:
use_zeroshot_previous_step = True
n = 8
top_n = 8

batch_size = 4
num_text_pairs = 10
num_epochs = 5

zeroshot_config['top_n'] = top_n
zeroshot_config['n_examples'] = n
zeroshot_config['exec_time'] = save_time

zeroshot_config["batch_size"] = batch_size
zeroshot_config["num_pairs"] = num_text_pairs
zeroshot_config["num_epochs"] = num_epochs

zeroshot_config['st_train_epochs'] = 10
zeroshot_config['st_train_batch_size'] = 20

split = "zeroshot" if use_zeroshot_previous_step else "fewshot"


In [23]:
zeroshot_previous_data = datasets_handler.getZeroshotPreviousData(which_dataset,class_col,top_n=top_n,exec_time=zeroshot_config['exec_time'])
raw_data_final, zeroshot_config['new_class_col'] = datasets_handler.mergeLabelingToDataset(raw_data,zeroshot_previous_data,class_col)

In [24]:
df_train, df_test = datasets_handler.splitDataset(raw_data_final,zeroshot_config)
train_dataset,test_dataset = datasets_handler.buildDatasetDict(df_train,df_test)
train_documents = datasets_handler.splitDocuments(df_train[data_col])
len(train_documents)

83


1985

In [25]:
# df_test

## Treinamento do Corpo do Modelo -- não descomentar

In [26]:
# model.fit(train_documents,batch_size=zeroshot_config['st_train_batch_size'],epochs=zeroshot_config['st_train_epochs'])

# Fine-Tuning (SetFit)

## Model

In [30]:
zeroshot_config['setfit_model']

'ricardo-filho/bert-base-portuguese-cased-nli-assin-2'

In [31]:
# setfit_model.model_body = model.embeddingModel

setfit_model = SetFitModel.from_pretrained(zeroshot_config['setfit_model'],
                                    use_differentiable_head=True,
                                    head_params={"out_features":len(zeroshot_config['classes'])})

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [45]:
# setfit_model.model_head.temperature = 1.0

In [46]:
# Create trainer
trainer = SetFitTrainer(
    model=setfit_model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    batch_size=zeroshot_config["batch_size"],
    num_iterations=zeroshot_config["num_pairs"], # Number of text pairs to generate for contrastive learning
    num_epochs=zeroshot_config["num_epochs"], # Number of epochs to use for contrastive learning
    column_mapping = {data_col: "text", 'class_code': "label"} # NÃO mudar
)
zeroshot_config

{'dataset': 'folhauol',
 'class_col': 'class',
 'data_col': 'text',
 'split': 'zeroshot',
 'method': 'dotproduct',
 'max_inferences': 1000,
 'classes': ['Poder e Política no Brasil',
  'Mercado',
  'Notícias de fora do Brasil',
  'Esporte',
  'Tecnologia',
  'Meio Ambiente',
  'Equilíbrio e Saúde',
  'Educação',
  'TV, Televisão e Entretenimento',
  'Ciência',
  'Turismo',
  'Comida'],
 'template': '{}',
 'random_state': 422,
 'top_n': 8,
 'n_examples': 8,
 'exec_time': '2023_04_07__15_56_16',
 'batch_size': 4,
 'num_pairs': 10,
 'num_epochs': 5,
 'st_train_epochs': 10,
 'st_train_batch_size': 20,
 'new_class_col': 'new_class',
 'setfit_model': 'ricardo-filho/bert-base-portuguese-cased-nli-assin-2'}

In [47]:
%%time
# Train and evaluate
# trainer.freeze() # Freeze the head
# trainer.train() # Train only the body


# Unfreeze the head and freeze the body -> head-only training
# zeroshot_config['keep_body_frozen_setfit'] = True

# Unfreeze the head and unfreeze the body -> end-to-end training
zeroshot_config['keep_body_frozen_setfit'] = False

print(zeroshot_config)


trainer.unfreeze(keep_body_frozen=zeroshot_config['keep_body_frozen_setfit'])


trainer.train(
    num_epochs=zeroshot_config["num_epochs"], # The number of epochs to train the head or the whole model (body and head)
    batch_size=zeroshot_config["batch_size"],
    body_learning_rate=1e-5, # The body's learning rate
    learning_rate=1e-2, # The head's learning rate
    l2_weight=0.1, # Weight decay on **both** the body and head. If `None`, will use 0.01.
)
# zeroshot_config['trainings_done'] += 1

Applying column mapping to training dataset
The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 512.


{'dataset': 'folhauol', 'class_col': 'class', 'data_col': 'text', 'split': 'zeroshot', 'method': 'dotproduct', 'max_inferences': 1000, 'classes': ['Poder e Política no Brasil', 'Mercado', 'Notícias de fora do Brasil', 'Esporte', 'Tecnologia', 'Meio Ambiente', 'Equilíbrio e Saúde', 'Educação', 'TV, Televisão e Entretenimento', 'Ciência', 'Turismo', 'Comida'], 'template': '{}', 'random_state': 422, 'top_n': 8, 'n_examples': 8, 'exec_time': '2023_04_07__15_56_16', 'batch_size': 4, 'num_pairs': 10, 'num_epochs': 5, 'st_train_epochs': 10, 'st_train_batch_size': 20, 'new_class_col': 'new_class', 'setfit_model': 'ricardo-filho/bert-base-portuguese-cased-nli-assin-2', 'keep_body_frozen_setfit': False}


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 14min 25s, sys: 8min 44s, total: 23min 9s
Wall time: 9min 14s


In [48]:
gc.collect()

y_pred = zeroberto.getPredictions(trainer)
# 5000 sentences --- 4min40s
# 10000 sentences ---

Running predictions on 996 sentences.


## Evaluation

In [49]:
try: print(setfit_all_metrics)
except: pass
setfit_all_metrics = evaluation_metrics.get_metrics(y_pred ,test_dataset["class_code"])
print(setfit_all_metrics)

/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'weighted': [{'accuracy': 0.3092369477911647}, {'precision': 0.28463838415115295}, {'recall': 0.3092369477911647}, {'f1': 0.2434155908262449}], 'macro': [{'accuracy': 0.3092369477911647}, {'precision': 0.28463838415115295}, {'recall': 0.3092369477911647}, {'f1': 0.2434155908262449}]}


In [50]:
# new_embeddingModel = trainer.model.model_body

## Save CL results

In [51]:
setfit_exec_time  = evaluation_metrics.saveResults(zeroshot_config,setfit_all_metrics)
print(zeroshot_config)

metrics_setfit_2023_04_07__16_12_23.csv
config_setfit_2023_04_07__16_12_23.csv
{'dataset': 'folhauol', 'class_col': 'class', 'data_col': 'text', 'split': 'zeroshot', 'method': 'dotproduct', 'max_inferences': 1000, 'classes': ['Poder e Política no Brasil', 'Mercado', 'Notícias de fora do Brasil', 'Esporte', 'Tecnologia', 'Meio Ambiente', 'Equilíbrio e Saúde', 'Educação', 'TV, Televisão e Entretenimento', 'Ciência', 'Turismo', 'Comida'], 'template': '{}', 'random_state': 422, 'top_n': 8, 'n_examples': 8, 'exec_time': '2023_04_07__15_56_16', 'batch_size': 4, 'num_pairs': 10, 'num_epochs': 5, 'st_train_epochs': 10, 'st_train_batch_size': 20, 'new_class_col': 'new_class', 'setfit_model': 'ricardo-filho/bert-base-portuguese-cased-nli-assin-2', 'keep_body_frozen_setfit': False}


## Save Model

In [ ]:
# model_path = "/Users/alealcoforado/Documents/Projetos/Modelos/{exec}".format(exec=setfit_exec_time)
# print(model_path)
# trainer.model._save_pretrained(save_directory=model_path)

/Users/alealcoforado/Documents/Projetos/Modelos/2023_03_09__14_05_21
